<a href="https://colab.research.google.com/github/py200041592/CEE2/blob/main/12_pandas_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas - Parte 2

## Aplicar funções em um DataFrame

`DataFrame.agg()` e `DataFrame.transform()` aplicam, respectivamente, uma função definida pelo usuário para reduzir ou propagar os resultados.

* `DataFrame.agg(func, axis=0)` é utilizada para aplicar funções por eixos.

* `DataFrame.transform(func, axis=0)` é utilizada para operar elemento a elemento.

Veja os exemplos:

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': [5, 6, np.nan, 8],
    'C': [10, 20, 30, 40]
})

# Aplicar uma única função em todas as colunas
result = df.agg('mean')
print(result)
# A    2.5
# B    6.333333
# C    25.0

# Aplicar múltiplas funções a todas as colunas

  ## Função personalizada para calcular a variação percentual
def amplitude(series):
    amp = float( series.max() - series.min() )
    return amp

result = df.agg(['sum', 'min', amplitude])
print("\n", result)
#                     A     B      C
# sum              10.0  19.0  100.0
# min               1.0   5.0   10.0
# amplitude         3.0   3.0   30.0

# Aplicar diferentes funções a diferentes colunas
result = df.agg({'A': 'sum', 'B': 'mean', 'C': 'max'})
print("\n")
print(result)
# A     10.0
# B      6.333333
# C     40.0


A     2.500000
B     6.333333
C    25.000000
dtype: float64

               A     B      C
sum        10.0  19.0  100.0
min         1.0   5.0   10.0
amplitude   3.0   3.0   30.0


A    10.000000
B     6.333333
C    40.000000
dtype: float64


In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': [5, 6, np.nan, 8],
    'C': [10, 20, 30, 40]
})

print("df:\n", df)

# Exemplo: padronizar os dados subtraindo de cada elemento o mínimo da coluna e
# dividindo pela pela amplitude de cada coluna
result = df.transform( lambda x: (x - x.min())/(x.max() - x.min()) )
print("\nresult:\n", result)
#     A     B     C
# 0   2  10.0   20
# 1   4  12.0   40
# 2   6   NaN   60
# 3   8  16.0   80

# Aplicar funções diferentes a cada coluna
result = df.transform({'A': lambda x: x + 10, 'B': lambda x: x.fillna(0), 'C': np.sqrt})
print("\nresult:\n", result)
#      A    B         C
# 0   11  5.0  3.162278
# 1   12  6.0  4.472136
# 2   13  0.0  5.477226
# 3   14  8.0  6.324555


df:
    A    B   C
0  1  5.0  10
1  2  6.0  20
2  3  NaN  30
3  4  8.0  40

result:
           A         B         C
0  0.000000  0.000000  0.000000
1  0.333333  0.333333  0.333333
2  0.666667       NaN  0.666667
3  1.000000  1.000000  1.000000

result:
     A    B         C
0  11  5.0  3.162278
1  12  6.0  4.472136
2  13  0.0  5.477226
3  14  8.0  6.324555


**Explicação**: `lambda` é uma forma de declarar um **função anônima**, geralmente utilizada em situações mais simples em que a função não será reaproveitada no futuro.

* sintaxe:
```python
lambda argumentos: expressão
```



### Exercício 1

Considere o seguinte `DataFrame`:
```python
dados = {
    "Região": ["Norte", "Norte", "Sul", "Sul", "Leste", "Leste"],
    "Produto": ["Maçã", "Banana", "Maçã", "Banana", "Maçã", "Banana"],
    "Vendas": [120, 200, 150, 300, 250, 180],
    "Lucro": [30, 50, 25, 70, 60, 40],
    "Desconto (%)": [5, 10, 0, 15, 5, 10],
}
df = pd.DataFrame(dados)
```
Então:

1. Use `DataFrame.agg()` para calcular, para cada coluna numérica do DataFrame:
  * A soma;
  * A média;
  * O valor máximo;

1. Use `DataFrame.transform()` para criar uma nova coluna chamada *Vendas Normalizadas*, que:
  * Subtraia a média das vendas de cada valor.
  * Divida pelo desvio padrão das vendas.
  
  Dica: Aplique a normalização (fórmula: $z = (x - média) / std$) diretamente sobre a coluna *Vendas*.


In [17]:
import pandas as pd
import numpy as np

dados = {
    "Região": ["Norte", "Norte", "Sul", "Sul", "Leste", "Leste"],
    "Produto": ["Maçã", "Banana", "Maçã", "Banana", "Maçã", "Banana"],
    "Vendas": [120, 200, 150, 300, 250, 180],
    "Lucro": [30, 50, 25, 70, 60, 40],
    "Desconto (%)": [5, 10, 0, 15, 5, 10],
}

ex1 = pd.DataFrame(dados)

resultado = ex1.agg({'Vendas': 'sum', 'Lucro': 'mean', 'Desconto (%)': 'max'})
print(resultado)

normalizados = ex1.transform({'Vendas': lambda x: (x - x.mean()) / x.std()})
ex1['Vendas Normalizadas'] = normalizados['Vendas']
print(f"\n {ex1}")

Vendas          1200.000000
Lucro             45.833333
Desconto (%)      15.000000
dtype: float64

   Região Produto  Vendas  Lucro  Desconto (%)  Vendas Normalizadas
0  Norte    Maçã     120     30             5            -1.211565
1  Norte  Banana     200     50            10             0.000000
2    Sul    Maçã     150     25             0            -0.757228
3    Sul  Banana     300     70            15             1.514456
4  Leste    Maçã     250     60             5             0.757228
5  Leste  Banana     180     40            10            -0.302891


## Contagem de valores

Pandas disponbiliza a função `count()` que pode ser aplicada para objetos do tipo `Series` e `DataFrame`. Esta função é usada para contar o número de valores **não nulos**, seja por linha ou coluna.

```python
DataFrame.count(axis=0, level=None, numeric_only=False)
```
1. `axis`:
  * 0 ou 'index' (padrão): Conta os valores não nulos por coluna.
  * 1 ou 'columns': Conta os valores não nulos por linha.

1. `level` (opcional):
  * Usado quando o DataFrame possui um índice hierárquico (MultiIndex).
  * Especifica o nível do índice para calcular os valores.

3. `numeric_only` (opcional):
  * Se True, considera apenas colunas (ou linhas) numéricas

 Veja o exemplo:


In [18]:
import pandas as pd
import numpy as np

# Criando um DataFrame de exemplo
df = pd.DataFrame({
    "A": [1, 2, np.nan, 4],
    "B": [np.nan, 2, 3, 4],
    "C": ["foo", "bar", np.nan, "baz"],
    "D": [np.nan, np.nan, np.nan, np.nan],
})

print("DataFrame:")
print(df)

# Contando valores não nulos em cada coluna
print("\nContagem por coluna:")
print(df.count())


DataFrame:
     A    B    C   D
0  1.0  NaN  foo NaN
1  2.0  2.0  bar NaN
2  NaN  3.0  NaN NaN
3  4.0  4.0  baz NaN

Contagem por coluna:
A    3
B    3
C    3
D    0
dtype: int64


O método `Series.value_counts()` em pandas conta a frequência de cada valor único em um objeto `Series`, ou seja, quantas vezes cada valor aparece.

> É uma ferramenta muito útil para análise de dados categóricos, pois oferece uma visão rápida da distribuição de valores.

In [19]:
import pandas as pd

# Criando uma Series
serie = pd.Series(["maçã", "banana", "laranja", "maçã", "banana", "maçã"])

print("Série original:")
print(serie)

# Contando valores únicos
print("\nContagem de valores:")
print(serie.value_counts())

# Frequência relativa
print("\nFrequência relativa:")
print(serie.value_counts(normalize=True))

Série original:
0       maçã
1     banana
2    laranja
3       maçã
4     banana
5       maçã
dtype: object

Contagem de valores:
maçã       3
banana     2
laranja    1
Name: count, dtype: int64

Frequência relativa:
maçã       0.500000
banana     0.333333
laranja    0.166667
Name: proportion, dtype: float64


## Métodos para *strings*

`Series` possui um conjunto de métodos para processar atributos do tipo `str`.

In [20]:
s = pd.Series(["A", "11B8", "99C", "Aaba", "Baca", np.nan, "CABA", "81dog", "cat"])
print( s.str.lower() )
print("\n")
print( s.str.findall("\d+") )

## no caso de dataframe, então deve ser aplicado por coluna
df = pd.DataFrame({"A": ["AA99", "asda21", "21asd23"],
                   "B": ["AA99", "asda21", "21asd23"]})

print("\ndf:\n", df)

df_transformed = df.agg(lambda col: col.str.findall(r"\d+"))
print("\n")
print(df_transformed)

0        a
1     11b8
2      99c
3     aaba
4     baca
5      NaN
6     caba
7    81dog
8      cat
dtype: object


0         []
1    [11, 8]
2       [99]
3         []
4         []
5        NaN
6         []
7       [81]
8         []
dtype: object

df:
          A        B
0     AA99     AA99
1   asda21   asda21
2  21asd23  21asd23


          A         B
0      [99]      [99]
1      [21]      [21]
2  [21, 23]  [21, 23]


## Concatenação

A concatenação de objetos do *Pandas* é feito com `concat()`:

In [21]:
df = pd.DataFrame(np.random.randn(10, 4))
df

,0,1,2,3
0,0.540544,1.396326,0.597960,0.196804
1,0.389461,-1.116041,-0.463192,0.468393
2,0.726404,-0.317965,-0.105111,1.779560
3,0.623002,-0.416391,0.140079,0.237046
4,2.215274,0.335986,-0.931562,-0.495075
5,0.031679,1.096130,0.845927,0.243280
6,1.325723,-0.490582,-0.908892,0.419722
7,-1.921500,1.088227,0.056384,0.877543
8,-0.968009,2.035591,-1.958814,0.891238
9,0.849085,-0.477073,0.291611,0.427027


In [22]:
# quebra em vários pedaços
pieces = (df[:3], df[3:7], df[7:]) ## tupla de DataFrames

# junção dos pedaços
pd.concat(pieces, axis=0)

,0,1,2,3
0,0.540544,1.396326,0.597960,0.196804
1,0.389461,-1.116041,-0.463192,0.468393
2,0.726404,-0.317965,-0.105111,1.779560
3,0.623002,-0.416391,0.140079,0.237046
4,2.215274,0.335986,-0.931562,-0.495075
5,0.031679,1.096130,0.845927,0.243280
6,1.325723,-0.490582,-0.908892,0.419722
7,-1.921500,1.088227,0.056384,0.877543
8,-0.968009,2.035591,-1.958814,0.891238
9,0.849085,-0.477073,0.291611,0.427027


## Exercício 2

Considere o seguinte `DataFrame`:
```python
dados = {
    "Cliente": ["Ana", "bruno", "Carlos", "Diana", "Eduarda", np.nan, "fábio", "Gabriela"],
    "Produto": ["Notebook", "Smartphone", "Notebook", "Tablet", "Notebook", "Smartphone", np.nan, "Tablet"],
    "Valor": [3000, 2000, 3000, 1500, 3000, 2000, 1500, 1500],
    "Data": ["2023-01-15", "2023-01-16", np.nan, "2023-02-10", "2023-02-10", "2023-01-16", "2023-02-15", "2023-02-16"],
}
df = pd.DataFrame(dados)
```
Então:

1. Qual é o número de valores não nulos em cada coluna do DataFrame?
  
  Dica: Utilize o método `count()`.

1. Qual produto foi vendido mais vezes?
  
  Dica: Utilize o método `value_counts()` para listar a quantidade de prudutos vendidos e `.idxmax()` para mostrar qual o produto mais vendito.
  
1. Passe a primeira letra do nome de cada cliente para maiúscula. Dica: Utilize o método `str.title()`.

1. Inclua o seguinte dados no final do `df`:
```python
df2 = pd.DataFrame({
    "Cliente": ["Jose", "Paula"],
    "Produto": ["Tablet", "Notebook"],
    "Valor": [1500, 4000],
    "Data": ["2023-03-01", "2023-03-02"],
})
```


In [25]:
dados = {
    "Cliente": ["Ana", "bruno", "Carlos", "Diana", "Eduarda", np.nan, "fábio", "Gabriela"],
    "Produto": ["Notebook", "Smartphone", "Notebook", "Tablet", "Notebook", "Smartphone", np.nan, "Tablet"],
    "Valor": [3000, 2000, 3000, 1500, 3000, 2000, 1500, 1500],
    "Data": ["2023-01-15", "2023-01-16", np.nan, "2023-02-10", "2023-02-10", "2023-01-16", "2023-02-15", "2023-02-16"],
}
ex2 = pd.DataFrame(dados)

nulos = ex2.count()
print(nulos)

mais_vendido = ex2['Produto'].value_counts().idxmax()
print(f"\n{mais_vendido}")

ex2['Cliente'] = ex2['Cliente'].str.title()
print(f"\n{ex2}")

df2 = pd.DataFrame({
 "Cliente": ["Jose", "Paula"],
 "Produto": ["Tablet", "Notebook"],
 "Valor": [1500, 4000],
 "Data": ["2023-03-01", "2023-03-02"],
})

ex2 = pd.concat([ex2, df2], ignore_index=True)
print(f"\n{ex2}")

Cliente    7
Produto    7
Valor      8
Data       7
dtype: int64

Notebook

    Cliente     Produto  Valor        Data
0       Ana    Notebook   3000  2023-01-15
1     Bruno  Smartphone   2000  2023-01-16
2    Carlos    Notebook   3000         NaN
3     Diana      Tablet   1500  2023-02-10
4   Eduarda    Notebook   3000  2023-02-10
5       NaN  Smartphone   2000  2023-01-16
6     Fábio         NaN   1500  2023-02-15
7  Gabriela      Tablet   1500  2023-02-16

    Cliente     Produto  Valor        Data
0       Ana    Notebook   3000  2023-01-15
1     Bruno  Smartphone   2000  2023-01-16
2    Carlos    Notebook   3000         NaN
3     Diana      Tablet   1500  2023-02-10
4   Eduarda    Notebook   3000  2023-02-10
5       NaN  Smartphone   2000  2023-01-16
6     Fábio         NaN   1500  2023-02-15
7  Gabriela      Tablet   1500  2023-02-16
8      Jose      Tablet   1500  2023-03-01
9     Paula    Notebook   4000  2023-03-02


## Junção de *dataframe*s

A função `merge()` habilita junções no estilo SQL entre colunas.

In [26]:
import pandas as pd

# DataFrame à esquerda
esquerda = pd.DataFrame({"produto": ["maçã", "pera", "banana", "uva"], "quantidade": [3, 5, 4, 2]})

# DataFrame à direita
direita = pd.DataFrame({"produto": ["pera", "banana", "maçã"], "preço": [4.50, 2.34, 3.25]})

print("esquerda:\n", esquerda)
print("\ndireita:\n", direita)

# 1) Mesclando os DataFrames
## O produto "uva" está presente em esquerda, mas não em direita.
## Por padrão esse produto é excluido por não ter correspondencia
resultado = pd.merge(esquerda, direita, on="produto")
print("\nResultado da junção 1:\n", resultado)

# 2) Mesclando os DataFrames (how='left')
## Neste caso, o valor na coluna "preço" para essa linha é NaN
resultado = pd.merge(esquerda, direita, on="produto", how='left')
print("\nResultado da junção 2:\n", resultado)

esquerda:
   produto  quantidade
0    maçã           3
1    pera           5
2  banana           4
3     uva           2

direita:
   produto  preço
0    pera   4.50
1  banana   2.34
2    maçã   3.25

Resultado da junção 1:
   produto  quantidade  preço
0    maçã           3   3.25
1    pera           5   4.50
2  banana           4   2.34

Resultado da junção 2:
   produto  quantidade  preço
0    maçã           3   3.25
1    pera           5   4.50
2  banana           4   2.34
3     uva           2    NaN


Outro exemplo, agora com 2 chaves.

In [27]:
import pandas as pd

# DataFrame à esquerda com mais um atributo (fornecedor)
esquerda = pd.DataFrame({
    "produto": ["maçã", "pera", "banana", "maçã", "banana"],
    "cor": ["vermelha", "verde", "amarela", "verde", "amarela"],
    "quantidade": [3, 5, 4, 6, 7],
    "fornecedor": ["Fornecedor A", "Fornecedor B", "Fornecedor C", "Fornecedor C", "Fornecedor A"]
})

# DataFrame à direita
direita = pd.DataFrame({
    "produto": ["maçã", "pera", "banana"],
    "cor": ["vermelha", "verde", "amarela"],
    "preço": [3.25, 4.50, 2.34]
})

print("esquerda:\n", esquerda)
print("\ndireita:\n", direita)

# Mesclando os DataFrames com mais de uma chave (produto, cor)
resultado = pd.merge(esquerda, direita, on=["produto", "cor"], how="left")
print("\nResultado da junção com múltiplas chaves (produto, cor) e fornecedor:\n", resultado)


esquerda:
   produto       cor  quantidade    fornecedor
0    maçã  vermelha           3  Fornecedor A
1    pera     verde           5  Fornecedor B
2  banana   amarela           4  Fornecedor C
3    maçã     verde           6  Fornecedor C
4  banana   amarela           7  Fornecedor A

direita:
   produto       cor  preço
0    maçã  vermelha   3.25
1    pera     verde   4.50
2  banana   amarela   2.34

Resultado da junção com múltiplas chaves (produto, cor) e fornecedor:
   produto       cor  quantidade    fornecedor  preço
0    maçã  vermelha           3  Fornecedor A   3.25
1    pera     verde           5  Fornecedor B   4.50
2  banana   amarela           4  Fornecedor C   2.34
3    maçã     verde           6  Fornecedor C    NaN
4  banana   amarela           7  Fornecedor A   2.34


## Agrupamento

Por “agrupamento” se refere a um processo que envolve um ou mais dos seguintes passos:

- **particionar** os dados em grupos basedos em algum critério;

- **aplicar** uma função em cada grupo independentemente;

- **combinar** os resultados em uma estrutura de dados.

In [28]:
import pandas as pd
import numpy as np

# Criando o DataFrame simulando vendas de banana e maça por regiões
df = pd.DataFrame(
    {
        "Produto": ["Maçã", "Banana", "Maçã", "Banana", "Maçã", "Banana", "Maçã", "Maçã"],
        "Região": ["Norte", "Norte", "Sul", "Sul", "Leste", "Leste", "Norte", "Leste"],
        "Vendas": np.abs(np.random.randn(8)),  # Valores numéricos aleatórios representando as vendas
        "Lucro": np.random.randn(8),   # Valores numéricos aleatórios representando o lucro
    }
)

print(df)

  Produto Região    Vendas     Lucro
0    Maçã  Norte  0.910971 -0.545467
1  Banana  Norte  0.183158  1.053056
2    Maçã    Sul  0.377249  0.095070
3  Banana    Sul  0.412831  0.375782
4    Maçã  Leste  0.414432  0.708129
5  Banana  Leste  0.923627  0.582965
6    Maçã  Norte  0.456816  0.440529
7    Maçã  Leste  1.292043  0.348246


Exemplo, agrupando por uma coluna, selecionando outras colunas e aplicando a função `sum()` aos dados resultantes:

In [29]:
print("\nAgrupamento por 'Produto' e soma de Vendas e Lucro:")
print(df.groupby("Produto")[["Vendas", "Lucro"]].sum())


Agrupamento por 'Produto' e soma de Vendas e Lucro:
           Vendas     Lucro
Produto                    
Banana   1.519615  2.011804
Maçã     3.451512  1.046508


Agrupar por múltiplas colunas forma um `MultiIndex`.

In [30]:
# Agrupando por Produto e Região e somando as colunas 'Vendas' e 'Lucro'
print("\nAgrupamento por 'Produto' e 'Região' e soma de Vendas e Lucro:")
print(df.groupby(["Produto", "Região"]).sum())


Agrupamento por 'Produto' e 'Região' e soma de Vendas e Lucro:
                  Vendas     Lucro
Produto Região                    
Banana  Leste   0.923627  0.582965
        Norte   0.183158  1.053056
        Sul     0.412831  0.375782
Maçã    Leste   1.706476  1.056375
        Norte   1.367787 -0.104938
        Sul     0.377249  0.095070


## Exercício 3

1. Para o código abaixo use a função `merge()` para combinar os dois DataFrames com base na coluna `Produto`.
  * E depois `how="outer"` (união de todos os dados).  
  * E depois `how="inner"` (interseção entre os DataFrames).

2. Para a ultima junção, faça um agrupamento pela coluna `Produto` e então selecione as colunas `Quantidade` e `Preço` e some elas.

```python
import pandas as pd

# DataFrame de produtos vendidos
vendas = pd.DataFrame({
    "Cliente": ["Ana", "Bruno", "Carlos", "Diana", "Eduarda", "Fábio", "Gabriela"],
    "Produto": ["Notebook", "Smartphone", "Notebook", "Tablet", "Notebook", "Mouse", "Smartphone"],
    "Data": ["2023-01-15", "2023-01-16", "2023-01-16", "2023-02-10", "2023-02-10", "2023-02-15", "2023-02-16"],
    "Quantidade": [1, 2, 1, 1, 3, 2, 1],
})

# DataFrame com os preços dos produtos
precos = pd.DataFrame({
    "Produto": ["Notebook", "Smartphone", "Tablet"],
    "Preço": [3000, 2000, 1500],
})

print("DataFrame de vendas:")
print(vendas)

print("\nDataFrame de preços:")
print(precos)
```

In [32]:
import pandas as pd

# DataFrame de produtos vendidos
vendas = pd.DataFrame({
    "Cliente": ["Ana", "Bruno", "Carlos", "Diana", "Eduarda", "Fábio", "Gabriela"],
    "Produto": ["Notebook", "Smartphone", "Notebook", "Tablet", "Notebook", "Mouse", "Smartphone"],
    "Data": ["2023-01-15", "2023-01-16", "2023-01-16", "2023-02-10", "2023-02-10", "2023-02-15", "2023-02-16"],
    "Quantidade": [1, 2, 1, 1, 3, 2, 1],
})

# DataFrame com os preços dos produtos
precos = pd.DataFrame({
    "Produto": ["Notebook", "Smartphone", "Tablet"],
    "Preço": [3000, 2000, 1500],
})

print("DataFrame de vendas:")
print(vendas)

print("\nDataFrame de preços:")
print(precos)

uniao = pd.merge(vendas, precos, on="Produto", how="outer")
print(f"\n{uniao}")

intersecao = pd.merge(vendas, precos, on="Produto", how="inner")
print(f"\n{intersecao}")

quant_preco = intersecao.groupby("Produto")[["Quantidade", "Preço"]].sum()
print(f"\n{quant_preco}")

DataFrame de vendas:
    Cliente     Produto        Data  Quantidade
0       Ana    Notebook  2023-01-15           1
1     Bruno  Smartphone  2023-01-16           2
2    Carlos    Notebook  2023-01-16           1
3     Diana      Tablet  2023-02-10           1
4   Eduarda    Notebook  2023-02-10           3
5     Fábio       Mouse  2023-02-15           2
6  Gabriela  Smartphone  2023-02-16           1

DataFrame de preços:
      Produto  Preço
0    Notebook   3000
1  Smartphone   2000
2      Tablet   1500

    Cliente     Produto        Data  Quantidade   Preço
0     Fábio       Mouse  2023-02-15           2     NaN
1       Ana    Notebook  2023-01-15           1  3000.0
2    Carlos    Notebook  2023-01-16           1  3000.0
3   Eduarda    Notebook  2023-02-10           3  3000.0
4     Bruno  Smartphone  2023-01-16           2  2000.0
5  Gabriela  Smartphone  2023-02-16           1  2000.0
6     Diana      Tablet  2023-02-10           1  1500.0

    Cliente     Produto        Data  Qu

## Reorganização

Para exemplos, vamos primeiramente criar um DataFrame para análise de dados relacionados a vendas e custos.

Este DataFrame será criado com um índice hierárquico de duas categorias:
* Categoria (Fruta, Legume, etc.)
* Métrica (Venda, Custo).

In [33]:
import pandas as pd
import numpy as np

# MultiIndex com dois níveis: Categoria e Métrica
arrays = [
    ["Fruta", "Fruta", "Legume", "Legume", "Bebida", "Bebida", "Grão", "Grão"],
    ["Venda", "Custo", "Venda", "Custo", "Venda", "Custo", "Venda", "Custo"],
]

index = pd.MultiIndex.from_arrays(arrays, names=["Categoria", "Métrica"])

# Criando um DataFrame com valores aleatórios para Vendas e Custos
df = pd.DataFrame(abs(np.random.randn(8, 3)), index=index, columns=["Janeiro", "Fevereiro", "Março"])
print("DataFrame Original:\n")
print(df)

DataFrame Original:

                    Janeiro  Fevereiro     Março
Categoria Métrica                               
Fruta     Venda    2.313884   0.331629  1.848829
          Custo    1.030051   2.816397  0.161157
Legume    Venda    0.289541   1.541124  0.352285
          Custo    0.265815   1.123567  1.477800
Bebida    Venda    1.824377   1.794947  0.449246
          Custo    0.724813   0.746716  1.709293
Grão      Venda    0.867377   0.763161  1.279894
          Custo    0.469518   0.350046  0.437504


### Empilhamento

* A função `stack()` modifica a distribuição das células para criar um formato vertical.

* A função `unstack()` faz o inverso do `stack()`, retorna o formato original.

  Como temos dois níveis de indices, também podemos utilizar:
  * `unstack(0)` para reorganiza pelo primeiro indice (Categoria) como colunas;
  * `unstack(1)` para reorganiza pelo segundo indice (Métrica) como colunas.  

In [34]:
# Empilhando as colunas para transformar em um formato mais vertical
stacked = df.stack()
print("\nDataFrame Empilhado (com stack):\n")
print(stacked)

# Desempilhando para reorganizar os dados de volta ao formato tabular
unstacked = stacked.unstack()
print("\n\nDataFrame Desempilhado (com unstack):\n")
print(unstacked)


DataFrame Empilhado (com stack):

Categoria  Métrica           
Fruta      Venda    Janeiro      2.313884
                    Fevereiro    0.331629
                    Março        1.848829
           Custo    Janeiro      1.030051
                    Fevereiro    2.816397
                    Março        0.161157
Legume     Venda    Janeiro      0.289541
                    Fevereiro    1.541124
                    Março        0.352285
           Custo    Janeiro      0.265815
                    Fevereiro    1.123567
                    Março        1.477800
Bebida     Venda    Janeiro      1.824377
                    Fevereiro    1.794947
                    Março        0.449246
           Custo    Janeiro      0.724813
                    Fevereiro    0.746716
                    Março        1.709293
Grão       Venda    Janeiro      0.867377
                    Fevereiro    0.763161
                    Março        1.279894
           Custo    Janeiro      0.469518
           

In [35]:
# Desempilhando com foco no nível 0 (Categoria)
unstacked_level0 = stacked.unstack(0)
print("\n\nDataFrame Desempilhado por 'Categoria' (nível 0):\n")
print(unstacked_level0)

# Desempilhando com foco no nível 1 (Métrica)
unstacked_level1 = stacked.unstack(1)
print("\n\nDataFrame Desempilhado por 'Métrica' (nível 1):\n")
print(unstacked_level1)




DataFrame Desempilhado por 'Categoria' (nível 0):

Categoria            Bebida     Fruta      Grão    Legume
Métrica                                                  
Custo   Janeiro    0.724813  1.030051  0.469518  0.265815
        Fevereiro  0.746716  2.816397  0.350046  1.123567
        Março      1.709293  0.161157  0.437504  1.477800
Venda   Janeiro    1.824377  2.313884  0.867377  0.289541
        Fevereiro  1.794947  0.331629  0.763161  1.541124
        Março      0.449246  1.848829  1.279894  0.352285


DataFrame Desempilhado por 'Métrica' (nível 1):

Métrica                 Custo     Venda
Categoria                              
Bebida    Janeiro    0.724813  1.824377
          Fevereiro  0.746716  1.794947
          Março      1.709293  0.449246
Fruta     Janeiro    1.030051  2.313884
          Fevereiro  2.816397  0.331629
          Março      0.161157  1.848829
Grão      Janeiro    0.469518  0.867377
          Fevereiro  0.350046  0.763161
          Março      0.437504  1

### Pivot tables

A função `pivot_table` permite:
* resumir as informações contidas em um DataFrame;
* personalizar a forma como os dados são organizados;
* utilizar agregações complexas e funções customizadas.

**Sintaxe:**

```python
pd.pivot_table(data, values=None, index=None, columns=None, aggfunc="mean",
 fill_value=None, margins=False, margins_name="All", dropna=True, observed=False, sort=True)
```

Principais argumentos:

* `data`: O DataFrame a partir do qual a tabela dinâmica será criada.
* `values`: A(s) coluna(s) cujos valores você deseja agregar.
* `index`: A(s) coluna(s) que formarão os índices da tabela.
* `columns`: A(s) coluna(s) que formarão as colunas da tabela.
* `aggfunc`: A função de agregação a ser usada. O padrão é "mean", mas você pode usar outras como "sum", "count", "max", "min", ou funções personalizadas. Pode ser uma lista de funções.
* `fill_value`: Valor para preencher células vazias ou NaN.
* `margins`: Inclui nas margens o total agregado.
* `observed`: Se `True`, então as variáveis categóricas ficam restritas apenas aos níveis observados.

Veja os exemplos.

In [36]:
import pandas as pd
import numpy as np

# Criando o DataFrame
dados = {
    "Categoria": ["Eletrônicos", "Eletrônicos", "Eletrodomésticos", "Eletrodomésticos", "Móveis", "Móveis"],
    "Produto": ["Notebook", "Smartphone", "Geladeira", "Fogão", "Sofá", "Cama"],
    "Vendas": [5000, 3000, 4000, 2500, 2000, 1500],
    "Ano": [2023, 2023, 2023, 2023, 2024, 2024]
}

df = pd.DataFrame(dados)
print(df)

          Categoria     Produto  Vendas   Ano
0       Eletrônicos    Notebook    5000  2023
1       Eletrônicos  Smartphone    3000  2023
2  Eletrodomésticos   Geladeira    4000  2023
3  Eletrodomésticos       Fogão    2500  2023
4            Móveis        Sofá    2000  2024
5            Móveis        Cama    1500  2024


**1. Agregando por uma única coluna:**

Vamos organizar as vendas por categoria usando pivot_table.

In [37]:
# Vendas totais por Categoria
tabela = pd.pivot_table(df, values="Vendas", index="Categoria", aggfunc="sum")
print(tabela)


                  Vendas
Categoria               
Eletrodomésticos    6500
Eletrônicos         8000
Móveis              3500


**2. Selecionando colunas:**

In [38]:
# Vendas totais por Categoria e Ano
tabela = pd.pivot_table(df, values="Vendas", index="Categoria",
                        columns="Ano", aggfunc="sum", fill_value=0)
print(tabela)

print("\n")

# Vendas totais por Categoria e Ano
tabela = pd.pivot_table(df, values="Vendas", index="Categoria",
                        columns=["Ano","Produto"], aggfunc="sum", fill_value=0)
print(tabela)


Ano               2023  2024
Categoria                   
Eletrodomésticos  6500     0
Eletrônicos       8000     0
Móveis               0  3500


Ano               2023                                2024      
Produto          Fogão Geladeira Notebook Smartphone  Cama  Sofá
Categoria                                                       
Eletrodomésticos  2500      4000        0          0     0     0
Eletrônicos          0         0     5000       3000     0     0
Móveis               0         0        0          0  1500  2000


**3. Adicionando margens (Totais):**

In [39]:
# Incluindo uma linha/coluna de total
tabela = pd.pivot_table(df, values="Vendas", index="Categoria", columns="Ano",
        aggfunc="sum", fill_value=0, margins=True, margins_name="Total Geral")
print(tabela)


Ano                2023  2024  Total Geral
Categoria                                 
Eletrodomésticos   6500     0         6500
Eletrônicos        8000     0         8000
Móveis                0  3500         3500
Total Geral       14500  3500        18000


**4. Pivot Table com múltiplos índices e colunas:**

In [40]:
# Pivot Table com múltiplos índices e colunas
tabela = pd.pivot_table(df, values="Vendas", index=["Categoria", "Produto"],
                        columns="Ano", aggfunc="sum", fill_value=0)
print(tabela)


Ano                          2023  2024
Categoria        Produto               
Eletrodomésticos Fogão       2500     0
                 Geladeira   4000     0
Eletrônicos      Notebook    5000     0
                 Smartphone  3000     0
Móveis           Cama           0  1500
                 Sofá           0  2000


### Exercício 4

Considere o seguinte conjunto de dados:
```python
import pandas as pd
import numpy as np

# DataFrame para os exercícios
dados = {
    "Produto": ["Notebook", "Notebook", "Smartphone", "Smartphone", "Tablet", "Tablet"],
    "Região": ["Norte", "Sul", "Norte", "Sul", "Norte", "Sul"],
    "Ano": [2023, 2023, 2024, 2024, 2023, 2024],
    "Vendas": [5000, 4500, 3000, 3500, 2000, 2500],
    "Lucro": [1000, 900, 800, 850, 300, 400],
}

df = pd.DataFrame(dados)
print(df)
```

Então:

1. Aplique `stack()` no DataFrame original e observe como ele transforma as colunas em um índice.

  Pergunta: Qual a principal diferença entre o DataFrame antes e depois de usar `stack()`?

2. A partir do resultado de `stack()`, use `unstack()` para reorganizar os dados. Experimente utilizar os níveis de índice (0, 1 ou nomeados).

3. Crie uma tabela dinâmica que mostre o lucro médio (Lucro) por "Produto" e "Região". Use `aggfunc="mean"`.

  Pergunta: Como a função de agregação afeta os dados?

4. Adicione margens (totais) à tabela dinâmica do exercício anterior, usando o parâmetro `margins=True`.   

In [44]:
import pandas as pd
import numpy as np

# DataFrame para os exercícios
dados = {
    "Produto": ["Notebook", "Notebook", "Smartphone", "Smartphone", "Tablet", "Tablet"],
    "Região": ["Norte", "Sul", "Norte", "Sul", "Norte", "Sul"],
    "Ano": [2023, 2023, 2024, 2024, 2023, 2024],
    "Vendas": [5000, 4500, 3000, 3500, 2000, 2500],
    "Lucro": [1000, 900, 800, 850, 300, 400],
}

df = pd.DataFrame(dados)
print(df)

coluna_indice = df.stack()
print(f"\n{coluna_indice}")

reorganizar = coluna_indice.unstack()
print(f"\n{reorganizar}")

tabela = pd.pivot_table(df, values="Lucro", index="Produto", columns="Região", aggfunc="mean")
print(f"\n{tabela}")

margens = pd.pivot_table(df, values="Lucro", index="Produto", columns="Região", aggfunc="mean", margins=True)
print(f"\n{margens}")

      Produto Região   Ano  Vendas  Lucro
0    Notebook  Norte  2023    5000   1000
1    Notebook    Sul  2023    4500    900
2  Smartphone  Norte  2024    3000    800
3  Smartphone    Sul  2024    3500    850
4      Tablet  Norte  2023    2000    300
5      Tablet    Sul  2024    2500    400

0  Produto      Notebook
   Região          Norte
   Ano              2023
   Vendas           5000
   Lucro            1000
1  Produto      Notebook
   Região            Sul
   Ano              2023
   Vendas           4500
   Lucro             900
2  Produto    Smartphone
   Região          Norte
   Ano              2024
   Vendas           3000
   Lucro             800
3  Produto    Smartphone
   Região            Sul
   Ano              2024
   Vendas           3500
   Lucro             850
4  Produto        Tablet
   Região          Norte
   Ano              2023
   Vendas           2000
   Lucro             300
5  Produto        Tablet
   Região            Sul
   Ano              2024
   Ve